<a href="https://colab.research.google.com/github/betancurmunera0411/Datos-micos/blob/main/metagenomics/notebooks/metagenomics_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Metagenomics Basic Analyses

## Metagenomics Data Analysis: Diversity and Ordination

This is a comprehensive notebook for performing a preliminary analysis of gut metagenomic data from the Human Microbiome Project 2 (HMP2) to explore microbial community structure, focusing on **alpha diversity** and **beta diversity** in relation to Inflammatory Bowel Disease (IBD) diagnosis (Crohn's Disease (**CD**), Ulcerative Colitis (**UC**), and **nonIBD** controls).

We will use the Python library **`scikit-bio`**, a bioinformatics package that provides functions for diversity analysis (alpha and beta) and ordination methods like Principal Coordinate Analysis (PCoA).

In [ ]:
!pip install scikit-bio

## Data Setup: Directory Creation and Downloads

These cells set up the working environment by creating necessary directories and downloading the raw data files from the HMP2 public repository.

1.  **Directory Creation:** Creates `metagenomics/data` and `metagenomics/results` folders to organize input and output files.
2.  **Data Download:**
      * Downloads a gzipped file containing the **taxonomic profiles** (`taxonomic_profiles.tsv.gz`), which is an abundance table where rows are microbial taxa and columns are samples. This file is then unzipped.
      * Downloads the **sample metadata** (`hmp2_metadata_2018-08-20.csv`), which contains information about each sample, including the participant's diagnosis.

In [ ]:
! mkdir metagenomics
! mkdir metagenomics/data
! mkdir metagenomics/results

In [ ]:
! wget https://g-227ca.190ebd.75bc.data.globus.org/ibdmdb/products/HMP2/MGX/2018-05-04/taxonomic_profiles.tsv.gz -O metagenomics/data/taxonomic_profiles.tsv.gz
! gunzip metagenomics/data/taxonomic_profiles.tsv.gz

In [ ]:
! wget https://g-227ca.190ebd.75bc.data.globus.org/ibdmdb/metadata/hmp2_metadata_2018-08-20.csv -O metagenomics/data/hmp2_metadata_2018-08-20.csv

## Data Loading, Inspection, and Filtering

The raw data is loaded into `pandas` DataFrames, the sample IDs are set as the index, and an initial filtering step is performed to prepare the data for analysis.

**Loading Data:** The taxonomic profile table (`data`) and the metadata table (`metadata`) are loaded and indexed by their respective IDs.

In [ ]:
import pandas as pd

data = pd.read_csv('metagenomics/data/taxonomic_profiles.tsv', sep='\t')
data = data.loc[:,~data.columns.duplicated()]
data = data.set_index('#SampleID')
print(data.head())

metadata = pd.read_csv('metagenomics/data/hmp2_metadata_2018-08-20.csv')
print(metadata.shape)
metadata = metadata.drop_duplicates(subset='External ID', keep='first')
print(metadata.shape)
metadata = metadata.set_index('External ID')
print(metadata.head())

**Note:** There is a lot of metadata available for these samples.

In [ ]:
metadata.columns.tolist()

**Important:** We will focus only on the diagnosis groups.

In [ ]:
groups = metadata[['diagnosis']].reset_index().drop_duplicates().set_index('External ID')

## Relative Abundance Comparison (Phylum Level) 🦠

To provide an intuitive, high-level view of taxonomic shifts, it is useful to visualize the relative abundance of the major microbial groups (e.g., Phyla) in each diagnostic group.

### Method: Taxonomic Aggregation and Stacked Bar Plot

This method aggregates the complex species-level data into broader taxonomic categories (Phyla).

1.  **Taxonomic Aggregation:** The code assumes the column names in your `data` contain the full taxonomic lineage (e.g., `k__Bacteria|p__Firmicutes|c__...`). It extracts the **Phylum level** by splitting the string.
2.  **Relative Abundance:** The absolute counts are converted to relative abundances (fractions of the total community) for each sample.
3.  **Group Averaging:** The average relative abundance for each Phylum is calculated for the three diagnostic groups (CD, UC, nonIBD).
4.  **Stacked Bar Plot:** A stacked bar plot is generated, where the height of the bar is $100\%$ and the segments represent the proportional contribution of each Phylum to the average community in that group.

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px

# This assumes your taxonomic data is annotated and can be aggregated.
# Assuming the first part of the column name (before the first '|') is the Phylum
phylum_df = data.T.copy()

# Simple aggregation example:
# If you have full taxonomy in columns like 'k__Bacteria|p__Firmicutes;...'
# Use string operations to extract the phylum level:
phylum_df.columns = phylum_df.columns.map(lambda x: x.split('|')[1] if len(x.split('|')) > 1 else 'NA')
phylum_df = phylum_df.drop('NA', axis=1)
phylum_grouped = phylum_df.groupby(by=phylum_df.columns, axis=1).sum()


# Convert to relative abundance (0 to 1)
phylum_rel_abun = phylum_grouped.div(phylum_grouped.sum(axis=1), axis=0)

# Merge with metadata for grouping
plot_rel_abun = phylum_rel_abun.merge(groups, left_index=True, right_index=True)

# 2. Plotting (e.g., showing average composition per group)
plot_data_melted = plot_rel_abun.groupby('diagnosis').mean().T.reset_index().melt(id_vars='index')


# Create the figure
fig = px.bar(
    plot_data_melted,
    x='diagnosis',
    y='value',
    color='index',
    title='Average Relative Abundance of Phyla by Diagnosis',
    labels={
        'diagnosis': 'Diagnosis Group',
        'value': 'Average Relative Abundance',
        'index': 'Phylum'
    },
    barmode='stack'
)

# To save the figure to a file Note: Not currently supported in Colab
#fig.write_image('metagenomics/results/relative_abundance_phylum.png', scale=3)

In [ ]:
fig

## Alpha Diversity Analysis (Shannon Index)

**Alpha diversity** is a metric used to assess the diversity within a single community (a single sample in this case). The **Shannon diversity index** quantifies the uncertainty in predicting the species identity of an individual randomly selected from a sample. Higher values indicate higher diversity and richness.

### Method: Shannon Index

The code uses the `scikit-bio` library to:

1.  **Calculate Shannon Diversity:** Apply the **'shannon' metric** from the `alpha_diversity` function to the sample-by-taxa abundance table.
2.  **Merge Data:** Join the calculated diversity scores with the sample diagnosis metadata, preparing the results for statistical comparison and plotting.

In [ ]:
from skbio.diversity.alpha import shannon

alpha_diversity = {}

for col in data.columns:
    alpha_diversity[col] = shannon(data[col], base=2)

alpha_diversity = pd.DataFrame.from_dict(alpha_diversity, orient='index')

In [ ]:
alpha_diversity.head()

In [ ]:
groups.head()

In [ ]:
alpha_diversity_df = pd.DataFrame({
    'Shannon_Diversity': alpha_diversity.iloc[:, 0],
    'Group': [groups.loc[i, 'diagnosis'] for i in alpha_diversity.index]
})
alpha_diversity_df.head()

## Visualization: Alpha Diversity Comparison

To visually compare the microbial diversity between the different disease groups (**nonIBD**, **CD** (Crohn's Disease), and **UC** (Ulcerative Colitis)), a **boxplot** is generated.

Boxplots are ideal for summarizing the distribution of a continuous variable (**Shannon Diversity**) across multiple categorical groups (**Diagnosis**). This plot will show the median, quartiles, and range of the alpha diversity for each diagnostic group, allowing for a visual assessment of whether IBD is associated with reduced or altered microbial diversity.

In [ ]:
from scipy.stats import mannwhitneyu


# Statistical test for Alpha Diversity between groups
stat, p_alpha = mannwhitneyu(
    alpha_diversity_df[alpha_diversity_df['Group'] == 'CD']['Shannon_Diversity'],
    alpha_diversity_df[alpha_diversity_df['Group'] == 'UC']['Shannon_Diversity']
)

print(f"\n## Alpha Diversity (Shannon) Test")
print(f"Mann-Whitney U P-value: {p_alpha:.4f}")

# Visualization of Alpha Diversity
import plotly.express as px

# Check if p_alpha is defined, otherwise use a placeholder value for the title
try:
    p_value = p_alpha
except NameError:
    p_value = 0.0001 # Placeholder value if p_alpha is not defined

# Create the figure
fig = px.box(
    alpha_diversity_df,
    x='Group',
    y='Shannon_Diversity',
    color='Group',
    title=f'Alpha Diversity (Shannon Index)<br>Mann-Whitney P={p_value:.4f}', # <br> creates a line break
    labels={
        'Group': 'Experimental Group',
        'Shannon_Diversity': 'Shannon Diversity'
    }
)
# To save the figure to a file Note: Not currently supported in Colab
#fig.write_image('metagenomics/results/alpha_diversity.png', scale=3)

In [ ]:
fig

## Differential Abundance Testing (Taxonomic)

**Differential abundance testing** goes beyond diversity metrics to identify **specific microbial taxa** that are statistically different in relative abundance between two or more groups (e.g., a specific *species* that is higher in CD patients compared to UC patients).

### Method: Mann-Whitney U Test and FDR Correction

This section uses the **Mann-Whitney U test** (a non-parametric test) to compare the distribution of relative abundances for each taxon between the two defined groups (Crohn's Disease vs. non-IBD control). The Mann-Whitney U test is ideal for this type of data as microbial abundances are often non-normally distributed and contain many zeros.

The code performs the following steps:

1.  **Define Groups:** Selects sample indices for the two comparison groups ('CD' vs. 'nonIBD').
2.  **Iterate and Test:** Iterates through every microbial taxon and calculates the **Mann-Whitney U test** (via `scipy.stats.ranksums` or `scipy.stats.mannwhitneyu`) to get a raw p-value for the difference in abundance.
3.  **Multiple Testing Correction (FDR):** The resulting raw p-values are then adjusted using the **Benjamini-Hochberg False Discovery Rate (FDR)** method (`smm.multipletests`). This is a crucial step to control the number of false positives that occur when performing thousands of individual tests.
4.  **Reporting:** Taxa with an adjusted p-value (FDR\_P\_Value) below the significance threshold (e.g., $0.05$) are reported as **significantly differentially abundant**.


In [ ]:
# We'll test for taxa significantly different between CD and UC
p_values = {}
for taxa_name in data.index:
    group_a_data = data.loc[taxa_name, list(set(data.columns).intersection(groups[groups['diagnosis'] == 'CD'].index))]
    group_b_data = data.loc[taxa_name, list(set(data.columns).intersection(groups[groups['diagnosis'] == 'UC'].index))]

    # Use Mann-Whitney U test (non-parametric, suitable for non-normal count data)
    stat, p_val = mannwhitneyu(group_a_data, group_b_data, alternative='two-sided')
    p_values[taxa_name] = p_val

# Convert results to DataFrame and apply multiple testing correction (e.g., Benjamini/Hochberg FDR)
diff_taxa_results = pd.DataFrame(list(p_values.items()), columns=['Taxa', 'P_Value'])
from statsmodels.sandbox.stats.multicomp import multipletests
_, corrected_p, _, _ = multipletests(diff_taxa_results['P_Value'], method='fdr_bh')
diff_taxa_results['FDR_P_Value'] = corrected_p

significant_taxa = diff_taxa_results[diff_taxa_results['FDR_P_Value'] < 0.05].sort_values('FDR_P_Value')

print(f"\n## Differential Abundance Test (Mann-Whitney U, FDR_P_Value < 0.05)")
print(f"Found {len(significant_taxa)} significantly different taxa:")
print(significant_taxa.head())
significant_taxa.to_csv('metagenomics/results/significant_taxa.csv', index=False)
print("-" * 40)



## Principal Component Analysis (PCA)

Although **PCoA** (using distances like Bray-Curtis) is standard, **Principal Component Analysis (PCA)** is a robust way to visualize clustering. Use the relative abundance data (Taxa as features, Samples as observations).

### Method: Principal Component Analysis (PCA)

**PCA** is an **ordination method** primarily used for **dimensionality reduction** on the *raw* or *normalized* **abundance data** itself, not on a distance matrix.

  * **Goal:** To find a set of orthogonal axes (Principal Components, or PCs) that capture the maximum possible **variance** in the *taxa abundances* across all samples.
  * **Interpretation:** PCA identifies co-varying groups of taxa that are responsible for the differences between samples. An ordination plot would show samples clustered by overall abundance profile, and a **biplot** could show which specific taxa (vectors) drive the separation.

In [ ]:
from sklearn.decomposition import PCA
X = data.T # Transpose: Samples are rows, Taxa are columns

# Run PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(X)
pca_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'], index=X.index)
pca_df['Group'] = [groups.loc[i, 'diagnosis'] for i in pca_df.index]

explained_variance = pca.explained_variance_ratio_
pc1_variance = explained_variance[0] * 100
pc2_variance = explained_variance[1] * 100
# Visualization of Beta Diversity
# Create the figure
fig = px.scatter(
    pca_df,
    x='PC1',
    y='PC2',
    color='Group',      # Equivalent to hue
    symbol='Group',     # Equivalent to style
    size_max=10,        # Plotly size is by area, not diameter (s=100 is large).
    opacity=0.8,        # Optional: Add slight transparency
    color_discrete_sequence=px.colors.qualitative.Vivid, # Use a color scheme similar to 'viridis' or a common qualitative one
    title='Beta Diversity Visualization (PCA)',
    labels={
        'PC1': f'PC1 ({pc1_variance:.2f}%)',
        'PC2': f'PC2 ({pc2_variance:.2f}%)',
        'Group': 'Group'
    }
)
# To save the figure to a file Note: Not currently supported in Colab
#fig.write_image('metagenomics/results/pca.png', scale=3)

In [ ]:
fig

## Beta Diversity Analysis: Bray-Curtis and PCoA

**Beta diversity** measures the difference in microbial community composition ***between*** samples. It is used to determine if the microbial community structure differs significantly across groups (e.g., between CD and nonIBD subjects).

### Method: Bray-Curtis Dissimilarity (Beta Diversity Metric)

The **Bray-Curtis dissimilarity** is an abundance-based **beta diversity metric** that quantifies the difference in microbial composition between two samples. A distance matrix is calculated, where each cell contains the Bray-Curtis distance (0 = identical, 1 = maximally different) between a pair of samples.

### Method: Principal Coordinate Analysis (PCoA)

**Principal Coordinate Analysis (PCoA)** (also known as Multidimensional Scaling, MDS) is an **ordination method** used to visualize the distances in the Bray-Curtis matrix in a low-dimensional space (typically 2D or 3D).

  * It finds a set of orthogonal axes (Principal Coordinates) that best represent the variation between samples based on the distance matrix.
  * The output reports the percentage of total variation explained by the first two axes (**PC1** and **PC2**), indicating how much of the original data's variability is captured in the 2D plot.


In [ ]:
import numpy as np
from skbio.stats.ordination import pcoa
from skbio.diversity import beta_diversity

# Calculate Beta-Diversity (Distance Matrix)
# PCoA requires a distance matrix, which is computed using a beta-diversity metric.
# Common metrics are Bray-Curtis, Jaccard, or UniFrac (if you have a phylogenetic tree).
print("--- Calculating Bray-Curtis Distance ---")

# Calculate Bray-Curtis distance
# 'braycurtis' is the metric, samples_taxa_df is the data, sample_ids are the indices
bray_curtis_dm = beta_diversity(
    metric='braycurtis',
    counts=data.T.values,
    ids=data.T.index
)

# bray_curtis_dm is a skbio.stats.distance.DistanceMatrix object
print(f"Distance Matrix Shape: {bray_curtis_dm.shape}\n")


# Perform PCoA
print("--- Performing PCoA ---")

# Run Principal Coordinate Analysis
# The input is the DistanceMatrix object
pcoa_results = pcoa(bray_curtis_dm)

# Access the proportion of variance explained by each axis
explained_variance = pcoa_results.proportion_explained
pc1_variance = explained_variance.iloc[0] * 100
pc2_variance = explained_variance.iloc[1] * 100

print(f"PC1 explains: {pc1_variance:.2f}%")
print(f"PC2 explains: {pc2_variance:.2f}%\n")

# Access the coordinates for plotting
pcoa_coords = pcoa_results.samples
pcoa_coords = pcoa_coords.iloc[:, [0, 1]].rename(columns={
    pcoa_coords.columns[0]: 'PC1',
    pcoa_coords.columns[1]: 'PC2'
})



## Visualization: PCoA Ordination Plot

The final step is to plot the sample coordinates from the PCoA (PC1 vs. PC2) to visualize how the overall microbial communities cluster.

### Interpretation

By coloring the points on the scatter plot according to the sample's **Diagnosis** (CD, UC, nonIBD), we can visually assess the **beta diversity**. If samples from the same diagnostic group cluster together and are separate from other groups, it suggests that the disease is a major factor shaping

In [ ]:
# Merge the PCoA coordinates with the sample metadata
plot_data = pcoa_coords.join(groups)
plot_column = 'diagnosis' # The metadata column to color the plot by

# Plotting with Matplotlib/Seaborn
print("--- Plotting PCoA Results ---")

fig = px.scatter(
    plot_data,
    x='PC1',
    y='PC2',
    color=plot_column,      # Equivalent to hue
    symbol=plot_column,     # Equivalent to style
    size_max=10,             # Plotly size is by area, not diameter (s=100 is large for Seaborn)
    opacity=0.8,             # Optional: add slight transparency to points
    title=f'PCoA of Samples (Bray-Curtis Distance), Colored by {plot_column}',
    labels={
        'PC1': f'PC1 ({pc1_variance:.2f}%)',
        'PC2': f'PC2 ({pc2_variance:.2f}%)',
        plot_column: plot_column # Ensures the legend title is correct
    }
)
# To save the figure to a file Note: Not currently supported in Colab
#fig.write_image('metagenomics/results/PCoA.png', scale=3)

In [ ]:
fig

## Statistical Validation: Beta Diversity (PERMANOVA) 🔬

While the PCoA plot provides a visual representation of how the microbial communities cluster, **PERMANOVA** (Permutational Multivariate Analysis of Variance) provides the statistical test to confirm if the observed differences between the groups are significant.

### Method: PERMANOVA (Permutational Multivariate Analysis of Variance)

**PERMANOVA** is the standard non-parametric test used in microbial ecology to test for significant differences in community composition (**beta diversity**) among groups.

  * **Input:** The **Bray-Curtis distance matrix** (calculated earlier) and the **categorical grouping variable** (the 'diagnosis' column).
  * **Mechanism:** It works by partitioning the variation in the distance matrix and comparing the observed variation *between* groups to the expected variation by randomly permuting the sample labels a large number of times (e.g., 999 permutations).
  * **Output:** A statistical test result, including an **$F$-statistic** and a **$p$-value**. A $p$-value less than $0.05$ indicates that the microbial community composition is significantly different across the diagnostic groups.

In [ ]:
# PERMANOVA requires the DistanceMatrix and the metadata
from skbio.stats.distance import permanova

# Run PERMANOVA on the Bray-Curtis distance matrix using the 'diagnosis' column
results_permanova = permanova(
    distance_matrix=bray_curtis_dm,
    grouping=groups['diagnosis'],
    permutations=999 # Use 999 permutations for robust p-value
)

print("\n--- PERMANOVA Test Results (Beta Diversity) ---")
print(results_permanova)

permanova_results_df = pd.DataFrame({
    'Test': ['PERMANOVA'],
    'F-Statistic': [results_permanova['test statistic']],
    'P-Value': [results_permanova['p-value']],
    'Permutations': [results_permanova['number of permutations']]
})
permanova_results_df.to_csv('metagenomics/results/permanova_results.csv', index=False)
# Check the P-value to determine if groups are significantly separated
if results_permanova['p-value'] < 0.05:
    print("\nResult: The microbial communities (Bray-Curtis) are significantly different across the diagnosis groups (CD, UC, nonIBD).")
else:
    print("\nResult: No significant difference found in microbial communities across groups.")

## Saving Outputs to Gooogle Drive

from google.colab import drive
drive.mount('/content/drive')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!cp -r metagenomics/ /content/drive/MyDrive/